In [0]:
## Program description: Multi-Class Binary Neural Network for Handwritten Digits ##
## Name of Author: Aditya Ranjan ##
## Link to problem: https://open.kattis.com/problems/mnist10class ##
## Training Set File can be found on the link above ##
## ----------------------------------------------------------------------------- ##


#Import Essential Libraries
import math
import random
import numpy as np
from tqdm import tqdm


#Creating Progress Bars for Training and Testing
trainingProgress = tqdm(total = 60000, position = 0)
testingProgress = tqdm(total = 60000, position = 0)


#Modified Sign Function for Perceptron Nodes
def sign(perceptronNode):
    if perceptronNode <= 0:
        return -1
    else:
        return 1


#Getting training data from the input file
inputFile = open("MultiClassTrain.txt")


#Creating Weights and Weight Sums Matrices (weight sums is the sum of weights from each training example)
#Note - Weights are binary (either -1 or 1)
weights = np.array([[random.choice([-1, 1]) for rowNum in range(51)] for columnNum in range(150)])
weightSums = np.array([[0 for rowNum in range(51)] for columnNum in range(150)])


#Indicates that first progress bar shows training progress
print("\n\n Training Progress:")


#Iterate through each training example
for trainingNum in range(60000):

    #Forward propagation - computes each layer of the network using the weights and inputs 
    inputLine = list(map(int, inputFile.readline().split(" "))) #list of values from current training example
    inputNodes = np.array(inputLine[0: 51]).reshape((51, 1)) #51 x values (compressed pixel information) of current training example
    trueOutput = inputLine[51] #y value of training example (indicates what number the handwritten digit is)

    perceptronNodes = np.sign(np.dot(weights, inputNodes)) #creates second layer of network - dot product of weights and inputs
    for rowNum in range(150): #loop to apply modified sign function to each perceptron node
        if perceptronNodes[rowNum] == 0:
            perceptronNodes[rowNum] -= 1

    sumNodes = np.array([sum(perceptronNodes[(rowNum * 15) : ((rowNum * 15) + 15)]) for rowNum in range(10)]).reshape(10, 1) #creates third layer of network - 10 nodes which are the sums of 15 consecutive perceptron nodes
    
    #Backward Propagation - calculates the amount of perceptron nodes and weights to change
    for sumNodeNum in range(10): #iterating through each sum node
        if sumNodeNum == trueOutput: #if the sum node matches the true output of the training example
            numPerceptronsToChange = (random.choice([7, 9, 11, 13, 15]) - sumNodes[sumNodeNum]) / 2 #calculates the number of perceptron nodes to change to push the current sum node value up to either 7, 9, 11, 13, or 15
            valToChange = -1 #changes needed amount of perceptron nodes and weights with a value of -1 to a value of 1
        elif sumNodes[sumNodeNum] > 5: #if the sum node doesn't match the true output of the training example and the sum node value is greater than zero
            numPerceptronsToChange = (random.choice([5, 7, 9, 11, 13, 15]) + sumNodes[sumNodeNum]) / 2 #calculates the number of perceptron nodes to change to push the current sum node value down to either -5, -7, -9, -11, -13, or -15
            valToChange = 1 #changes needed amount of perceptron nodes and weights with a value of 1 to a value of -1
        if sumNodeNum == trueOutput or sumNodes[sumNodeNum] > 5: #doesn't change perceptron nodes and weights for sum nodes that don't match true output or have values less than 5
            numPerceptronsChanged = 0
            for perceptronNum in range((sumNodeNum * 15), ((sumNodeNum * 15) + 15)): #iterates through 15 perceptron nodes for respective sum node
                if numPerceptronsChanged == numPerceptronsToChange:
                    break  #breaks out of loop as soon as needed number of perceptron nodes have been changed
                if perceptronNodes[perceptronNum] == valToChange: #changes perceptron node if the value is to be changed
                    if sumNodeNum == trueOutput: #calculation for sum node that matches the true output
                        numToFlip = math.ceil(((1 - int(np.sum((weights[perceptronNum] * inputNodes.T), axis = 1))) / 2)) #calculates number of weights to change based on dot product of weights and inputs
                    else: #calculation for sum node that doesn't match the true output
                        numToFlip = math.ceil(int(np.sum((weights[perceptronNum] * inputNodes.T), axis = 1)) / 2) #calculates number of weights to change based on dot product of weights and inputs

                    #Gradient Descent - changes necessary amount of weights which in turn changes the perceptron nodes and sum nodes as needed
                    numFlipped = 0
                    weightsIndex = random.choice([inputNum for inputNum in range(51)]) #starts process at a random index 
                    weightsCounter = random.choice([-1, 1]) #either iterates forward or backward randomly
                    while True:
                        if numFlipped == numToFlip:
                            break #breaks out of loop as soon as needed number of weights have been changed
                        if (weights[perceptronNum][weightsIndex] * inputNodes[weightsIndex]) == valToChange: #changes weight if the value is to be changed
                            weights[perceptronNum][weightsIndex] *= -1 #flips the weight value
                            numFlipped += 1 #updates the number of weights changed
                        weightsIndex = (weightsIndex + weightsCounter) % 51 #calculates the next index based on counter and current index
                    numPerceptronsChanged += 1 #updates the number of perceptron nodes changed

    weightSums += weights #adds weights to weight sums

    trainingProgress.update(1) #updates training progress bar


#Applies modified sign function to each weight sum
for rowNum in range(150): 
    weightSums[rowNum] = np.array(list(map(sign, weightSums[rowNum].tolist())))


#Prints trained weights as a list of strings
outputList = []
for rowNum in range(150):
  outputList.append(' '.join(map(str, weightSums[rowNum])))
print("\n\nTrained Weights:")
print(outputList)


#Indicates that second progress bar shows testing progress
print("\n Testing Progress:")


#Testing the trained weights on the training examples
accuracyCount = 0 #variable to count number of right predictions
inputFile = open("MultiClassTrain.txt") #opening training set file
for trainingNum in range(60000): #iterating through each training example

    #Forward Propagation with trained weights (same as forward propagation for training)
    inputLine = list(map(int, inputFile.readline().split(" ")))
    inputNodes = np.array(inputLine[0: 51]).reshape((51, 1))
    trueOutput = inputLine[51]
    perceptronNodes = np.sign(np.dot(weightSums, inputNodes))
    for rowNum in range(150):
        if perceptronNodes[rowNum] == 0:
            perceptronNodes[rowNum] -= 1
    sumNodes = np.array([sum(perceptronNodes[(rowNum * 15) : ((rowNum * 15) + 15)]) for rowNum in range(10)]).reshape(10, 1)

    #Checking the calculated prediction
    if int(np.argmax(sumNodes)) == trueOutput:
        accuracyCount += 1 #updates the accuracy count if the prediction is right

    testingProgress.update(1) #updates testing progress bar

print("\nThe accuracy of the trained weights on the training set is " + str(accuracyCount / 600) + "%") #prints percent accuracy of trained weights on the training set

  0%|          | 240/60000 [00:00<00:51, 1162.67it/s]



 Training Progress:


  0%|          | 278/60000 [00:45<522:12:48, 31.48s/it]



Trained Weights:
['1 -1 1 -1 -1 -1 -1 -1 -1 1 -1 1 -1 1 -1 -1 -1 -1 1 -1 1 1 -1 1 1 1 -1 -1 -1 -1 1 -1 1 1 1 1 1 1 1 -1 1 1 1 -1 1 1 -1 -1 1 1 -1', '1 1 1 -1 1 -1 -1 -1 -1 1 1 1 -1 -1 1 -1 1 -1 -1 -1 1 -1 -1 1 -1 1 1 1 -1 -1 1 1 -1 1 -1 1 1 1 1 1 1 1 1 -1 1 1 -1 1 1 1 1', '1 1 1 -1 1 -1 -1 -1 -1 1 1 1 -1 1 1 -1 -1 -1 1 -1 1 1 -1 1 -1 1 -1 1 -1 -1 1 1 1 1 1 1 1 1 -1 1 1 1 1 1 1 -1 -1 1 1 -1 -1', '1 -1 1 -1 1 -1 -1 -1 -1 1 -1 1 -1 1 1 1 -1 1 1 -1 -1 1 -1 -1 -1 1 1 1 -1 -1 1 -1 1 1 -1 1 1 1 -1 -1 1 1 -1 1 1 1 -1 1 1 1 -1', '1 -1 1 -1 1 -1 -1 -1 -1 -1 -1 1 -1 1 1 -1 1 1 1 1 1 -1 -1 -1 -1 1 1 1 1 -1 1 1 1 1 1 1 1 1 -1 -1 1 1 1 1 1 1 -1 -1 1 -1 -1', '1 1 1 -1 1 -1 -1 -1 -1 1 -1 1 -1 1 1 -1 -1 -1 -1 1 -1 1 -1 -1 -1 1 -1 1 -1 -1 1 1 1 1 1 1 1 1 -1 -1 1 1 1 1 1 1 -1 1 1 1 1', '1 -1 1 -1 1 -1 -1 1 1 1 -1 1 -1 1 1 -1 1 -1 -1 -1 1 1 -1 -1 1 1 1 1 -1 -1 1 1 -1 1 -1 -1 1 1 -1 -1 1 1 -1 1 1 1 -1 1 1 1 1', '1 1 1 -1 1 -1 -1 1 -1 1 -1 -1 -1 1 1 -1 -1 -1 1 -1 -1 -1 -1 1 1 1 1 1 1 -1 1 1 1 1 -1 1 1 1 -

100%|█████████▉| 59975/60000 [01:06<00:00, 2635.59it/s]


The accuracy of the trained weights on the training set is 72.07333333333334%
